In [2]:
# Part 1 - Loading data and data cleaning
# Loading dataset
import numpy as np
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('data/user_data.csv')

for col in dataset.columns:
    if col not in ['user_id', 'domain_name']:
        col_mean = np.nanmean(dataset[col], axis=0)
        dataset[col].fillna(col_mean,inplace=True)
        
# Replace with average age
X = dataset.iloc[:, 1:17].values
y = dataset.iloc[:, 17].values

In [3]:
# Encoding categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 15] = labelencoder_X_1.fit_transform(X[:, 15])

In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

/Users/richardchen/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [88]:
# Part 2 - Make ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialize ANN
classifier = Sequential()

# First hidden layer with 10% dropout
classifier.add(Dense(activation="relu", input_dim=16, units=8, kernel_initializer="uniform"))
classifier.add(Dropout(rate=0.1))

# The second hidden layer with 10% dropout
classifier.add(Dense(activation="relu", units=8, kernel_initializer="uniform"))
classifier.add(Dropout(rate=0.1))

# Adding the output layer
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

# Compiling the ANN
classifier.compile(
    optimizer='adam', # efficient SGD
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fit the data with the ANN model with selection of epochs (For Stochastic gradient decent) and data size to process
classifier.fit(
    x=X_train,
    y=y_train,
    batch_size=10, # How much data to process at once
    epochs=100 # Run through 100x SGD
)

/Users/richardchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=16, units=8, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app
/Users/richardchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
/Users/richardchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=8, kernel_initializer="uniform")`
/Users/richardchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
/Users/richardchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="unifor

Epoch 1/100
52213/52213 [==============================] - 8s 145us/step - loss: 0.1321 - acc: 0.9674
Epoch 2/100
52213/52213 [==============================] - 7s 142us/step - loss: 0.0711 - acc: 0.9799
Epoch 3/100
52213/52213 [==============================] - 8s 144us/step - loss: 0.0678 - acc: 0.9807
Epoch 4/100
52213/52213 [==============================] - 8s 147us/step - loss: 0.0665 - acc: 0.9815
Epoch 5/100
52213/52213 [==============================] - 7s 142us/step - loss: 0.0667 - acc: 0.9814
Epoch 6/100
52213/52213 [==============================] - 8s 149us/step - loss: 0.0658 - acc: 0.9813
Epoch 7/100
52213/52213 [==============================] - 8s 144us/step - loss: 0.0651 - acc: 0.9817
Epoch 8/100
52213/52213 [==============================] - 8s 146us/step - loss: 0.0640 - acc: 0.9820
Epoch 9/100
52213/52213 [==============================] - 8s 148us/step - loss: 0.0637 - acc: 0.9822
Epoch 10/100
52213/52213 [==============================] - 8s 153us/step - loss: 

In [89]:
# Part 3 - Making the prediction and print accuracy
y_pred = classifier.predict(X_test)

# If y_pred is > 0.5 then true, else false
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[8183   69]
 [ 230 8923]]


In [6]:
# Part 4 - Evaluating, improving and tuning the ANN using K-fold cross validation
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Keras classifier wrapper
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
    # Initialize ANN
    classifier = Sequential()

    # First hidden layer with 10% dropout
    classifier.add(Dense(activation="relu", input_dim=16, units=8, kernel_initializer="uniform"))
    classifier.add(Dropout(rate=0.1))

    # The second hidden layer with 10% dropout
    classifier.add(Dense(activation="relu", units=8, kernel_initializer="uniform"))
    classifier.add(Dropout(rate=0.1))

    # Adding the output layer
    classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

    # Compiling the ANN
    classifier.compile(
        optimizer='adam', # efficient SGD
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return classifier

# The difference is only in the training part
classifier = KerasClassifier(
    build_fn=build_classifier,
    batch_size=10,
    nb_epoch=100
)

# Apply k-fold
accuracies = cross_val_score(
    estimator=classifier,
    X=X_train,
    y=y_train,
    cv=10, # 10-fold cv
    n_jobs=-1, # CPU to use for estimation
)

# Compute the variance and mean accuracy
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
5221/5221 [==============================] - 1s 205us/step
Epoch 1/1
Epoch 1/1
5221/5221 [==============================] - 1s 113us/step


In [8]:
print(mean)
print(variance)

0.9796794070019125
0.0015707282567079363


In [9]:
# Part 5 - Parameter Tuning using GridSearch
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    # Initialize ANN
    classifier = Sequential()

    # First hidden layer with 10% dropout
    classifier.add(Dense(activation="relu", input_dim=16, units=8, kernel_initializer="uniform"))
    classifier.add(Dropout(rate=0.1))

    # The second hidden layer with 10% dropout
    classifier.add(Dense(activation="relu", units=8, kernel_initializer="uniform"))
    classifier.add(Dropout(rate=0.1))

    # Adding the output layer
    classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

    # Compiling the ANN
    classifier.compile(
        optimizer=optimizer, # efficient SGD
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return classifier

# The difference is only in the training part
classifier = KerasClassifier(build_fn=build_classifier)

# Create a dict of hyperparameters to optimize
parameters = {
    # Tune batch size, epoch, optimizer
    'batch_size': [25, 32],
    'nb_epoch': [100, 500],
    'optimizer': ['adam', 'rmsprop']
}

# Implement GridSearch
grid_search = GridSearchCV(
    estimator=classifier,
    param_grid=parameters,
    scoring='accuracy',
    cv=10
)

# Fit gridsearch to training set
grid_search = grid_search.fit(
    X_train,
    y_train
)

# Capture best param and accuracy
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/1
46991/46991 [==============================] - 3s 64us/step - loss: 0.1941 - acc: 0.9539
Epoch 1/1
46991/46991 [==============================] - 3s 67us/step - loss: 0.2251 - acc: 0.9465
Epoch 1/1
46991/46991 [==============================] - 3s 67us/step - loss: 0.1925 - acc: 0.9446
Epoch 1/1
46992/46992 [==============================] - 3s 73us/step - loss: 0.1885 - acc: 0.9481
Epoch 1/1
46992/46992 [==============================] - 4s 81us/step - loss: 0.1958 - acc: 0.9460
Epoch 1/1
46992/46992 [==============================] - 3s 69us/step - loss: 0.1931 - acc: 0.9484
Epoch 1/1
46992/46992 [==============================] - 4s 87us/step - loss: 0.2036 - acc: 0.9343
Epoch 1/1
46992/46992 [==============================] - 4s 75us/step - loss: 0.1948 - acc: 0.9468
Epoch 1/1
46992/46992 [==============================] - 3s 72us/step - loss: 0.2002 - acc: 0.9504
Epoch 1/1
46992/46992 [==============================] - 4s 77us/step - loss: 0.1936 - acc: 0.9481
Epoch 1/1


In [10]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'nb_epoch': 500, 'optimizer': 'rmsprop'}
0.9788941451362687
